In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features

fig_dir='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Saildrone/'

icefile='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Ice Present.xlsx'

data_dir = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission_combined/'



## read in ice present file

This file is a combination of the NOAA ice file & own analysis of data based on STD of salinity and SST

In [ ]:
ds = pd.read_excel(icefile, sheet_name='data',index_col=0)
ds_ice = ds.to_xarray()

## check data

Just print out the data and look at the dates to make sure start and stop look right

In [ ]:
ds_ice.sel(usv=1036).start_time

## Read in all the cruise full data

- swap dims from obs to time
- I checked & time = for both trajectories

In [ ]:
ds = xr.open_mfdataset(data_dir+'*.nc',combine='nested',concat_dim='trajectory').load()

#ds = ds.rename({'latitude':'lat','longitude':'lon'})
#ds['time']=ds.time[0,:]
#ds = ds.swap_dims({'obs':'time'})
#calculate some variables
#ds['wspd_MEAN']=np.sqrt(ds.UWND_MEAN**2+ds.VWND_MEAN**2)
#ds['wdir_MEAN']=np.arctan2(ds.VWND_MEAN,ds.UWND_MEAN)*180./np.pi
#tem=sw.dens0(ds.SAL_SBE37_MEAN,ds.TEMP_SBE37_MEAN)
#ds['density_MEAN']=xr.DataArray(tem,dims=('trajectory','time'),coords={'trajectory':ds.trajectory,'time':ds.time})
#tem=sw.alpha(ds.SAL_SBE37_MEAN,ds.TEMP_SBE37_MEAN,ds.BARO_PRES_MEAN*0) #pressure =0 at surface
#ds['alpha_MEAN']=xr.DataArray(tem,dims=('trajectory','time'),coords={'trajectory':ds.trajectory,'time':ds.time})
#tem=sw.beta(ds.SAL_SBE37_MEAN,ds.TEMP_SBE37_MEAN,ds.BARO_PRES_MEAN*0) #pressure =0 at surface
#ds['beta_MEAN']=xr.DataArray(tem,dims=('trajectory','time'),coords={'trajectory':ds.trajectory,'time':ds.time})

#get rid of bad salinity values
ds_masked=ds.where(ds.SAL_SBE37_MEAN>20)
ds['SAL_SBE37_MEAN']=ds_masked['SAL_SBE37_MEAN']



In [ ]:
ds

# Create ice mask based on photo/data

In [ ]:
ice_mask=np.zeros(ds.TEMP_AIR_MEAN.shape)
#ice_mask = xr.DataArray(ice_mask,coords=[ds.trajectory,ds.time],dims=['trajectory','time'],
#                                           attrs={'long_name':'ice mask 1=ice'})
ice_mask = xr.DataArray(ice_mask,coords=[ds.trajectory,ds.time],dims=['trajectory','time'],
                                           attrs={'long_name':'ice mask 1=ice'})
ds['ice_mask']=ice_mask
for iusv in range(2):
    ice =  ds_ice.sel(usv=int(ds.trajectory[iusv].data))
    ds_tem = ds.isel(trajectory=iusv)
    for i in range(ice.usv.size):
        print(iusv,ice.start_time[i].data)
        cond = (ds_tem.time>=ice.start_time[i]) & (ds_tem.time<=ice.stop_time[i])
        ds_tem.ice_mask[cond]=1
    ds['ice_mask'][iusv,:]=ds_tem.ice_mask

## Plot all the Variables 

In [ ]:
for var in ds.variables:
    if 'MEAN' in var:
        print(var)
        ax=ds[var][0,:].plot(linewidth=.5,color='r',label='1036')
        ax=ds[var][1,:].plot(linewidth=.5,color='b',label='1037')
        plt.legend()
        plt.title('')
        plt.savefig(fig_dir+'figs/'+var+'.png')
        plt.clf()

## Plot SSTs on map to show where vehicles whent

In [ ]:
for i in range(2):
    fig = plt.figure(figsize=(8,12))
    ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=180.0))  # create a set of axes with Mercator projection
    ds2 = ds.isel(trajectory=i)
    im=ax.scatter(ds2.lon,ds2.lat,c=ds2.TEMP_SBE37_MEAN,s=.15,transform=ccrs.PlateCarree(),label=ds.trajectory[i].data,cmap='jet')
    ax.coastlines(resolution='10m')                
    ax.set_extent([-180,-160,50,77])
    ax.legend()
    cax = fig.add_axes([0.45, 0.17, 0.3, 0.02])
    cbar = fig.colorbar(im,cax=cax, orientation='horizontal')
    cbar.set_label('SST ($^\deg$C)')
    fig.savefig(fig_dir+'figs/map_nasa'+str(i).zfill(2)+'_data.png')    
fig = plt.figure(figsize=(8,12))
ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=180.0))  # create a set of axes with Mercator projection
for i in range(2):
    ds2 = ds.isel(trajectory=i)
    im=ax.scatter(ds2.lon,ds2.lat,c=ds2.TEMP_SBE37_MEAN,s=.15,transform=ccrs.PlateCarree(),label=ds.trajectory[i].data,cmap='jet')
    ax.coastlines(resolution='10m')                
    ax.set_extent([-180,-160,50,77])
    ax.legend()
    cax = fig.add_axes([0.45, 0.17, 0.3, 0.02])
    cbar = fig.colorbar(im,cax=cax, orientation='horizontal')
    cbar.set_label('SST ($^\deg$C)')
    fig.savefig(fig_dir+'figs/map_nasa_data.png')

## make plot air sea tem difference

In [ ]:
for i in range(2):
    fig = plt.figure(figsize=(8,12))
    ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=180.0))  # create a set of axes with Mercator projection
    ds2 = ds.isel(trajectory=i)
    rdif=ds2.TEMP_AIR_MEAN-ds2.TEMP_SBE37_MEAN
    im=ax.scatter(ds2.lon,ds2.lat,c=rdif,s=.15,vmin=-3,vmax=3,transform=ccrs.PlateCarree(),label=ds.trajectory[i].data,cmap='seismic')
    ax.coastlines(resolution='10m')                
    ax.set_extent([-180,-160,50,77])
    ax.legend()
    cax = fig.add_axes([0.45, 0.17, 0.3, 0.02])
    cbar = fig.colorbar(im,cax=cax, orientation='horizontal')
    cbar.set_label('$\Delta$ T (air=sea) ($^\deg$C)')
    fig.savefig(fig_dir+'figs/map_nasa'+str(i).zfill(2)+'_data_airseadif.png')    
fig = plt.figure(figsize=(8,12))
ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=180.0))  # create a set of axes with Mercator projection
for i in range(2):
    ds2 = ds.isel(trajectory=i)
    rdif=ds2.TEMP_AIR_MEAN-ds2.TEMP_SBE37_MEAN
    im=ax.scatter(ds2.lon,ds2.lat,c=rdif,s=.15,vmin=-3,vmax=3,transform=ccrs.PlateCarree(),label=ds.trajectory[i].data,cmap='seismic')
    ax.coastlines(resolution='10m')                
    ax.set_extent([-180,-160,50,77])
    ax.legend()
    cax = fig.add_axes([0.45, 0.17, 0.3, 0.02])
    cbar = fig.colorbar(im,cax=cax, orientation='horizontal')
    cbar.set_label('$\Delta$ T (air=sea) ($^\deg$C)')
    fig.savefig(fig_dir+'figs/map_nasa_data_airseadif.png')

In [ ]:
fig = plt.figure(figsize=(5,5))
rdif=ds.TEMP_AIR_MEAN-ds.TEMP_SBE37_MEAN
sdif=rdif.stack(z=('trajectory','time'))
plt.hist(sdif,bins=np.arange(-6,6,.1))#,rwidth=0.25)
plt.xlabel('$\Delta$ T (air-sea)')
plt.ylabel('Num. obs')
plt.xticks(rotation=45)
fig.savefig(fig_dir+'figs/nasa_air_hist.png', dpi=300)


## Look at any wind speed or time of day dependence on differences between two inflow 0.6m bulk SSTs

In [ ]:
#make diruanl plot
ds2=ds#.isel(trajectory=0)
xlon=ds2.lon
tdif=ds2.TEMP_CTD_RBR_MEAN-ds2.TEMP_SBE37_MEAN
time_offset_to_lmt=(xlon/360.)*24.*60
ds2['tlmt']=ds2.lon
for i in range(2):
    ds2['tlmt'][i,:]= ds2.time.data+time_offset_to_lmt[i,:]*np.timedelta64(1,'m')# dt.timedelta(seconds=1)

In [ ]:
tdif=ds2.TEMP_CTD_RBR_MEAN-ds2.TEMP_SBE37_MEAN
fig,(ax1,ax2) =plt.subplots(1,2,figsize=(12,8))
for i in range(2):
    cs=ax1.scatter(ds2.wspd_MEAN[i,:],tdif[i,:],c=ds2.time.dt.hour,s=.5)
ax1.set(xlabel='Wind Speed (ms$^{-1}$)', ylabel='RBR - SBE4 SST (K)')
ax1.set_xlim(0,15)
cbar = fig.colorbar(cs,orientation='horizontal',ax=ax1)
cbar.set_label('GMT Time (hrs)')
for i in range(2):
    cs2=ax2.scatter(ds2.time.dt.hour,tdif[i,:],c=ds2.wspd_MEAN[i,:],s=.5)
ax2.set(xlabel='GMT (hr)')
cbar = fig.colorbar(cs2,orientation='horizontal',ax=ax2)
cbar.set_label('Wind Speed (ms$^{-1}$)')
fig.savefig(fig_dir+'figs/temp_buld_dw_data.png')

In [ ]:
ds2=ds
fig,(ax1,ax2) =plt.subplots(1,2,figsize=(12,8))
tdif=ds2.TEMP_CTD_RBR_MEAN-ds2.sea_water_temperature_01_mean
for i in range(2):
    cs=ax1.scatter(ds2.wspd_MEAN[i,:],tdif[i,:],c=ds2.time.dt.hour,s=.5)
ax1.set(xlabel='Wind Speed (ms$^{-1}$)', ylabel='RBR - SBE56 SST (K)')
ax1.set_xlim(0,15)
cbar = fig.colorbar(cs,orientation='horizontal',ax=ax1)
cbar.set_label('GMT Time (hrs)')
tdif=ds2.TEMP_SBE37_MEAN-ds2.sea_water_temperature_01_mean
for i in range(2):
    cs=ax2.scatter(ds2.wspd_MEAN[i,:],tdif[i,:],c=ds2.time.dt.hour,s=.5)
ax2.set(xlabel='Wind Speed (ms$^{-1}$)', ylabel='RBR - SBE56 SST (K)')
ax2.set_xlim(0,15)
cbar = fig.colorbar(cs,orientation='horizontal',ax=ax2)
cbar.set_label('GMT Time (hrs)')
fig.savefig(fig_dir+'figs/temp_comparesbe56_ctds_data.png')

In [ ]:
tdif=ds2.TEMP_CTD_RBR_MEAN-ds2.TEMP_SBE37_MEAN
cond=(ds.ice_mask==0)
tdif2=tdif.where(cond)
tdif[0,:].plot()
tdif2[0,:].plot()

In [ ]:
for i in range(2):
    fig = plt.figure(figsize=(8,12))
    ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=180.0))  # create a set of axes with Mercator projection
    ds2 = ds.isel(trajectory=i)
    rdif=ds2.TEMP_CTD_RBR_MEAN-ds2.TEMP_SBE37_MEAN
    im=ax.scatter(ds2.lon,ds2.lat,c=rdif,s=.15,vmin=-.5,vmax=.5,transform=ccrs.PlateCarree(),label=ds.trajectory[i].data,cmap='seismic')
    ax.coastlines(resolution='10m')                
    ax.set_extent([-180,-160,50,77])
    ax.legend()
    cax = fig.add_axes([0.45, 0.17, 0.3, 0.02])
    cbar = fig.colorbar(im,cax=cax, orientation='horizontal')
    cbar.set_label('$\Delta$ T (air=sea) ($^\deg$C)')
    fig.savefig(fig_dir+'figs/map_nasa'+str(i).zfill(2)+'_data_RBR_SBE_dif.png')    

In [ ]:
i=0
fig = plt.figure(figsize=(8,12))
ds2 = ds.isel(trajectory=i)
rdif=ds2.TEMP_CTD_RBR_MEAN-ds2.TEMP_SBE37_MEAN
rdif.plot()

## Make differences

In [ ]:
ds['tdif_temp']=ds.TEMP_CTD_RBR_MEAN-ds.TEMP_SBE37_MEAN
ds.tdif_temp.attrs=ds.TEMP_CTD_RBR_MEAN.attrs
ds.tdif_temp.attrs['long_name']='$\Delta$'+'(RBR - SBE) '+ds.tdif_temp.attrs['long_name']

ds['tdif_sal']=ds.SAL_RBR_MEAN-ds.SAL_SBE37_MEAN
ds.tdif_sal.attrs=ds.SAL_RBR_MEAN.attrs
ds.tdif_sal.attrs['long_name']='$\Delta$'+'(RBR - SBE) '+ds.tdif_sal.attrs['long_name']

ds['tdif_O2']=ds.O2_CONC_RBR_MEAN-ds.O2_CONC_SBE37_MEAN
ds.tdif_O2.attrs=ds.O2_CONC_RBR_MEAN.attrs
ds.tdif_O2.attrs['long_name']='$\Delta$'+'(RBR - SBE) '+ds.tdif_O2.attrs['long_name']

ds['tdif_Chlor']=ds.CHLOR_RBR_MEAN-ds.CHLOR_WETLABS_MEAN
ds.tdif_Chlor.attrs=ds.CHLOR_RBR_MEAN.attrs
ds.tdif_Chlor.attrs['long_name']='$\Delta$'+'(RBR - Wetlabs) '+ds.tdif_Chlor.attrs['long_name']


In [ ]:
print('CTD TEMP')
tdif=ds.tdif_temp
print(tdif.mean('time').data,tdif.std('time').data,sum(np.isfinite(tdif[0,:])).data,sum(np.isfinite(tdif[1,:])).data)
print('SAL')
tdif=ds.tdif_sal
print(tdif.mean('time').data,tdif.std('time').data,sum(np.isfinite(tdif[0,:])).data,sum(np.isfinite(tdif[1,:])).data)
print('O2 CONC')
tdif=ds.tdif_O2
print(tdif.mean('time').data,tdif.std('time').data,sum(np.isfinite(tdif[0,:])).data,sum(np.isfinite(tdif[1,:])).data)
print('Chlor')
tdif=ds.tdif_Chlor
print(tdif.mean('time').data,tdif.std('time').data,sum(np.isfinite(tdif[0,:])).data,sum(np.isfinite(tdif[1,:])).data)


## Plot differences

In [ ]:
for var in ds.variables:
    if 'tdif' in var:
        print(var)
        ax=ds[var][0,:].plot(linewidth=.5,color='r',label='1036')
        ax=ds[var][1,:].plot(linewidth=.5,color='b',label='1037')
        plt.legend()
        plt.title('')
        plt.savefig(fig_dir+'figs/'+var+'.png')
        plt.clf()